In [1]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import datetime as dt

import re, string
import nltk
import contractions
from nltk.corpus import stopwords

from api_keys import api_key

In [2]:
def request_song_info(artist_id, page_number):
    base_url = f'https://api.genius.com'
    headers = {f'Authorization': f'Bearer {api_key}'}
    search_url = f'{base_url}/artists/{artist_id}/songs?per_page=40&page={page_number}&sort=popularity'
    response = requests.get(search_url, headers=headers)

    return response

In [3]:
jay_dict = {}
beyonce_dict = {}
destiny_dict = {}
artist_dict = {2 : jay_dict, 498 : beyonce_dict, 2831 : destiny_dict}
        
for artist_id, artist_dict in artist_dict.items():      

    for page in range (1, 20):
        response = request_song_info(artist_id, page)
        json = response.json()
        remote_song_info = None

        for song in json['response']['songs']:
            if str(artist_id) in str(song['primary_artist']['id']):
                remote_song_info = song

                if remote_song_info:
                    song_url = remote_song_info['url']

                if len(artist_dict) == 400:
                    break

                else:
                    artist_dict.update({song['title'].lower() : song_url})

In [4]:
print(f'the jay_z_dict contains {len(jay_dict)} songs.')
print(f'the beyonce_dict contains {len(beyonce_dict)} songs.')
print(f'the destiny_dict contains {len(destiny_dict)} songs.')

the jay_z_dict contains 400 songs.
the beyonce_dict contains 308 songs.
the destiny_dict contains 152 songs.


In [5]:
def scrap_song_url(song_url, date_index):
    page = requests.get(song_url)
    html = BeautifulSoup(page.text, 'html.parser')
    release_date = dt.datetime.strptime(html.find_all('span', {'class': 'metadata_unit-info metadata_unit-info--text_only'})[date_index].get_text(), '%B %d, %Y').strftime('%Y%m%d')
    lyrics = html.find('div', class_='lyrics').get_text()

    return release_date, lyrics

In [6]:
relationship_start = scrap_song_url(jay_dict["'03 bonnie & clyde"], 1)[0]
print(int(relationship_start))

20021010


In [7]:
pre_jay_dict = {}
post_jay_dict = {}

for song, song_url in jay_dict.items():
    try:
        if int(scrap_song_url(song_url, 1)[0]) >= int(relationship_start):
            if len(post_jay_dict) == 50:
                continue
            else:
                post_jay_dict.update({song : [scrap_song_url(song_url, 1)[0], song_url, scrap_song_url(song_url, 1)[1]]})
        else:
            if len(pre_jay_dict) == 50:
                continue
            else:
                pre_jay_dict.update({song : [scrap_song_url(song_url, 1)[0], song_url, scrap_song_url(song_url, 1)[1]]})

    except:
        try:
            if int(scrap_song_url(song_url, 0)[0]) >= int(relationship_start):
                if len(post_jay_dict) == 50:
                    continue
                else:
                    post_jay_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})
            else:
                if len(pre_jay_dict) == 50:
                    continue
                else:
                    pre_jay_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})                
                
        except:
            print(f'FAILED on {song}')

FAILED on dead presidents iii
FAILED on history
FAILED on 44 four's
FAILED on a million and one questions (remix)
FAILED on blow the whistle
FAILED on 30 something (remix)
FAILED on open letter part ii (remix)
FAILED on pump it up freestyle
FAILED on if i can't (freestyle)
FAILED on when the money goes
FAILED on addicted to the game
FAILED on 2002 funk flex freestyle
FAILED on dead or alive, pt. 1
FAILED on early this morning
FAILED on hustle hard
FAILED on on the rock (remix)
FAILED on rap city freestyle
FAILED on purchases  basquiat painting 'mecca' for $4.5m
FAILED on appearance on real time with bill maher
FAILED on rehab (remix)
FAILED on analyze this
FAILED on flow
FAILED on hawaiian sophie
FAILED on thoughts on trayvon martin, president obama, race
FAILED on ranking his own albums
FAILED on the dynasty: roc la familia [credits]
FAILED on maybach music 2 (remix)
FAILED on magna carta app permissions
FAILED on excuse me miss again (remix)
FAILED on can't knock the hustle (remix)
F

In [8]:
print(len(pre_jay_dict))
print(len(post_jay_dict))

50
50


In [9]:
pre_beyonce_dict = {}
post_beyonce_dict = {}

for song, song_url in beyonce_dict.items():
    try:
        if int(scrap_song_url(song_url, 1)[0]) >= int(relationship_start):
            if len(post_beyonce_dict) == 50:
                continue
            else:
                post_beyonce_dict.update({song : [scrap_song_url(song_url, 1)[0],  song_url, scrap_song_url(song_url, 1)[1]]})
        else:
            if len(pre_beyonce_dict) == 50:
                continue
            else:
                pre_beyonce_dict.update({song : [scrap_song_url(song_url, 1)[0], song_url, scrap_song_url(song_url, 1)[1]]})

    except:
        try:
            if int(scrap_song_url(song_url, 0)[0]) >= int(relationship_start):
                if len(post_beyonce_dict) == 50:
                    continue
                else:
                    post_beyonce_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})
            else:
                if len(pre_beyonce_dict) == 50:
                    continue
                else:
                    pre_beyonce_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})                
                
        except:
            print(f'FAILED on {song}')

FAILED on bow down
FAILED on beyoncé [tracklist]
FAILED on hello
FAILED on roc
FAILED on wake up
FAILED on denial (poem)
FAILED on flaws and all
FAILED on resentment (live at on the run tour)
FAILED on i been on (remix)
FAILED on i go gets out
FAILED on standing on the sun (unfinished version)
FAILED on rise up
FAILED on god made you beautiful
FAILED on still in love (kissing you)
FAILED on beautiful liar (freemasons mix)
FAILED on satellites
FAILED on ghost
FAILED on one night only (disco)
FAILED on if looks could kill (you would be dead)
FAILED on sexy lil' thug (remix)
FAILED on yours and mine
FAILED on apology to fans
FAILED on runway
FAILED on i can't take no more
FAILED on apathy
FAILED on hard to say goodbye
FAILED on the formation world tour
FAILED on what is it gonna be
FAILED on me, myself & i (remix)
FAILED on cards never lie
FAILED on if i were a boy / you oughta know (live at the "52nd grammy awards")
FAILED on fighting temptation
FAILED on slow love
FAILED on letter to mi

In [10]:
print(len(pre_beyonce_dict))
print(len(post_beyonce_dict))

6
50


In [11]:
pre_destiny_dict = {}

for song, song_url in destiny_dict.items():
    try:
        if int(scrap_song_url(song_url, 1)[0]) < int(relationship_start):
            if len(pre_destiny_dict) == 50:
                continue
            else:
                pre_destiny_dict.update({song : [scrap_song_url(song_url, 1)[0], song_url, scrap_song_url(song_url, 1)[1]]})
    except:
        try:
            if int(scrap_song_url(song_url, 0)[0]) < int(relationship_start):
                if len(pre_destiny_dict) == 50:
                    continue
                else:
                    pre_destiny_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})
        except:
            print(f'FAILED on {song}')

FAILED on opera of the bells
FAILED on what's it gonna be
FAILED on i can't go for that
FAILED on why you actin'
FAILED on my man
FAILED on silent night
FAILED on jumpin', jumpin' (remix extended version)
FAILED on got's my own
FAILED on love me (not)
FAILED on proud family
FAILED on dot
FAILED on this love
FAILED on a "dc" christmas medley
FAILED on 2 step
FAILED on this christmas
FAILED on winter paradise
FAILED on hold my beer
FAILED on soldier (album extended version)
FAILED on o' holy night
FAILED on no more rainy days
FAILED on platinum bells
FAILED on gospel medley: you've been so good/now behold the lamb/jesus loves me/t
FAILED on i'm through with love
FAILED on bills, bills, bills (maurice's xclusive livegig mix)
FAILED on something for me
FAILED on before i let go
FAILED on life like this
FAILED on bills, bills, bills - #1's edit
FAILED on like dat
FAILED on sugar daddy
FAILED on you and me
FAILED on heart
FAILED on let's do us
FAILED on no no no (part 2 - extended remix)
FAI

In [12]:
print(len(pre_destiny_dict))

50


In [13]:
pre_jay_df = pd.DataFrame.from_dict(pre_jay_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics', ])
post_jay_df = pd.DataFrame.from_dict(post_jay_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics'])
pre_beyonce_df = pd.DataFrame.from_dict(pre_beyonce_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics'])
post_beyonce_df = pd.DataFrame.from_dict(post_beyonce_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics'])
pre_destiny_df = pd.DataFrame.from_dict(pre_destiny_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics'])

In [14]:
stop = stopwords.words('english')

def clean_text(artist_df):
    artist_df = artist_df.apply(lambda x: x.replace('\n', ' '))
    artist_df = artist_df.apply(lambda x: re.sub('\[[^]]*\]', '', x))
    artist_df = artist_df.apply(lambda x: x.lower())
    artist_df = artist_df.apply(lambda x: contractions.fix(x))
    artist_df = artist_df.apply(lambda x: re.sub(r'[^\w\s]','', x))
    artist_df = artist_df.apply(lambda x: [item for item in x.split() if item not in stop])
    return artist_df

In [15]:
pre_jay_df.lyrics = clean_text(pre_jay_df.lyrics)
post_jay_df.lyrics = clean_text(post_jay_df.lyrics)
pre_beyonce_df.lyrics = clean_text(pre_beyonce_df.lyrics)
post_beyonce_df.lyrics = clean_text(post_beyonce_df.lyrics)
pre_destiny_df.lyrics = clean_text(pre_destiny_df.lyrics)

In [16]:
pre_beyonce_df = pd.concat([pre_beyonce_df, pre_destiny_df])

In [17]:
pre_jay_df['unique_words'] = pre_jay_df.lyrics.apply(lambda x: len(set(x)))
pre_jay_df['word_count'] = pre_jay_df.lyrics.apply(lambda x: len(x))
pre_jay_df['percent_unique'] = pre_jay_df.unique_words / pre_jay_df.word_count

post_jay_df['unique_words'] = post_jay_df.lyrics.apply(lambda x: len(set(x)))
post_jay_df['word_count'] = post_jay_df.lyrics.apply(lambda x: len(x))
post_jay_df['percent_unique'] = post_jay_df.unique_words / post_jay_df.word_count

pre_beyonce_df['unique_words'] = pre_beyonce_df.lyrics.apply(lambda x: len(set(x)))
pre_beyonce_df['word_count'] = pre_beyonce_df.lyrics.apply(lambda x: len(x))
pre_beyonce_df['percent_unique'] = pre_beyonce_df.unique_words / pre_beyonce_df.word_count

post_beyonce_df['unique_words'] = post_beyonce_df.lyrics.apply(lambda x: len(set(x)))
post_beyonce_df['word_count'] = post_beyonce_df.lyrics.apply(lambda x: len(x))
post_beyonce_df['percent_unique'] = post_beyonce_df.unique_words / post_beyonce_df.word_count

In [18]:
pre_jay_df['years'] = pd.to_datetime(pre_jay_df['release_date'], format='%Y%m%d') - pd.to_datetime(relationship_start, format= '%Y%m%d')
pre_jay_df['years'] = pre_jay_df['years'].dt.days / 365

post_jay_df['years'] = pd.to_datetime(post_jay_df['release_date'], format='%Y%m%d') - pd.to_datetime(relationship_start, format= '%Y%m%d')
post_jay_df['years'] = post_jay_df['years'].dt.days / 365

pre_beyonce_df['years'] = pd.to_datetime(pre_beyonce_df['release_date'], format='%Y%m%d') - pd.to_datetime(relationship_start, format= '%Y%m%d')
pre_beyonce_df['years'] = pre_beyonce_df['years'].dt.days / 365

post_beyonce_df['years'] = pd.to_datetime(post_beyonce_df['release_date'], format='%Y%m%d') - pd.to_datetime(relationship_start, format= '%Y%m%d')
post_beyonce_df['years'] = post_beyonce_df['years'].dt.days / 365

In [19]:
lyric_data = {'sj_years' : pre_jay_df.years.values.tolist(), 
               'sj_percent_unique' : pre_jay_df.percent_unique.values.tolist(),
               'sj_word_count' : pre_jay_df.word_count.values.tolist(),
               'sj_song': pre_jay_df.index.tolist(),
               'sj_song_url': pre_jay_df.song_url.values.tolist(),
                'mj_years' : post_jay_df.years.values.tolist(), 
               'mj_percent_unique' : post_jay_df.percent_unique.values.tolist(),
               'mj_word_count' : post_jay_df.word_count.values.tolist(),
               'mj_song': post_jay_df.index.tolist(),
               'mj_song_url': post_jay_df.song_url.values.tolist(),
              'sb_years' : pre_beyonce_df.years.values.tolist(), 
               'sb_percent_unique' : pre_beyonce_df.percent_unique.values.tolist(),
               'sb_word_count' : pre_beyonce_df.word_count.values.tolist(),
               'sb_song': pre_beyonce_df.index.tolist(),
               'sb_song_url': pre_beyonce_df.song_url.values.tolist(),
                'mb_years' : post_beyonce_df.years.values.tolist(), 
               'mb_percent_unique' : post_beyonce_df.percent_unique.values.tolist(),
               'mb_word_count' : post_beyonce_df.word_count.values.tolist(),
               'mb_song': post_beyonce_df.index.tolist(),
               'mb_song_url': post_beyonce_df.song_url.values.tolist()}

In [20]:
lyric_data

{'sj_years': [-2.9945205479452053,
  -1.0794520547945206,
  -1.0794520547945206,
  -0.4246575342465753,
  -2.786301369863014,
  -1.0794520547945206,
  -6.298630136986302,
  -4.032876712328767,
  -6.295890410958904,
  -6.641095890410959,
  -1.9808219178082191,
  -1.0794520547945206,
  -1.0794520547945206,
  -6.295890410958904,
  -4.032876712328767,
  -1.021917808219178,
  -6.641095890410959,
  -6.123287671232877,
  -6.295890410958904,
  -4.934246575342466,
  -1.0794520547945206,
  -6.295890410958904,
  -0.8301369863013699,
  -5.5479452054794525,
  -1.9424657534246574,
  -6.295890410958904,
  -4.032876712328767,
  -4.934246575342466,
  -1.0794520547945206,
  -6.545205479452055,
  -4.934246575342466,
  -1.0794520547945206,
  -6.295890410958904,
  -1.0794520547945206,
  -2.652054794520548,
  -1.9424657534246574,
  -6.295890410958904,
  -1.0794520547945206,
  -3.9205479452054797,
  -4.934246575342466,
  -0.09863013698630137,
  -2.786301369863014,
  -6.295890410958904,
  -1.9424657534246574,